In [107]:
import requests
import datetime
import time
import pandas as pd
import numpy as np

In [11]:
r = requests.get('https://api.pro.coinbase.com/products/BTC-USD/candles?start=2020-12-27T00:00:00Z&end=2020-12-28T00:00:00Z&granularity=3600')
r.json()

[[1609113600, 26086.19, 26955.44, 26255.16, 26845.64, 1418.01702094],
 [1609110000, 26205.11, 26677.72, 26493.96, 26258.65, 1276.38185369],
 [1609106400, 26255, 26637.69, 26390.97, 26493.96, 950.25761812],
 [1609102800, 25772.55, 26444.65, 26184.83, 26388.94, 2844.36442243],
 [1609099200, 26134.79, 27070, 27058.39, 26194.55, 1839.56535711],
 [1609095600, 26951.01, 27274, 27024.2, 27058.39, 804.29465546],
 [1609092000, 26526, 27025, 26767.36, 27024.07, 1223.29475663],
 [1609088400, 26636.12, 27056.81, 27056.81, 26767.36, 1186.85176376],
 [1609084800, 26580, 27446.7, 27347.7, 27051.87, 2693.20303375],
 [1609081200, 27000, 27459.67, 27309.24, 27347.69, 1489.41402411],
 [1609077600, 27220.33, 27749.51, 27490.85, 27307.34, 1564.49125579],
 [1609074000, 27032.84, 27550, 27202.41, 27495.48, 1139.20603384],
 [1609070400, 26362.67, 27623.19, 27623.19, 27189.51, 2359.82878026],
 [1609066800, 26809.76, 28387, 27803.51, 27612.27, 2587.05855359],
 [1609063200, 27666.96, 27869.5, 27790, 27803.47, 38

In [75]:
def get_coinbase_data(start, end, granularity=86400, product_id="BTC-USD"):
    assert start >= datetime.datetime(2015, 7, 20)
    request_url = "https://api.pro.coinbase.com/products/{}/candles?start={}&end={}&granularity={}".format(product_id, start.isoformat(), end.isoformat(), granularity)
    r = requests.get(request_url)
    return r.json()

In [135]:
start = datetime.datetime(2015, 7, 20)
end = datetime.datetime.now()

def fetch_historic_data():
    curr_start = start
    curr_end = curr_start + datetime.timedelta(days=10)
    df = pd.DataFrame()
    while curr_start < end:
        data = get_coinbase_data(curr_start, curr_end, 3600)
        df = df.append(pd.DataFrame(np.array(data)[::-1]))
        
        curr_start = curr_end
        curr_end = curr_start + datetime.timedelta(days=10)
        time.sleep(0.4)
    return df

historic_df = fetch_historic_data()
historic_df.columns = ["time", "low", "high", "open", "close", "volume"]

In [139]:
historic_df = historic_df.drop_duplicates(subset=['time'])
historic_df.to_csv("../data/bitcoin_prices_v3.csv")
pd.read_csv("../data/bitcoin_prices_v3.csv")

,Unnamed: 0,time,low,high,open,close,volume
0,0,1.437426e+09,277.92,278.00,277.98,278.00,59.260456
1,1,1.437430e+09,277.37,278.50,278.00,278.14,346.995411
2,2,1.437433e+09,278.15,280.00,278.31,280.00,376.627552
3,3,1.437437e+09,279.38,281.00,279.96,280.89,288.177976
4,4,1.437440e+09,280.35,281.15,280.89,280.36,380.933023
...,...,...,...,...,...,...,...
47651,194,1.609121e+09,26881.99,27137.31,27042.66,27066.33,855.323479
47652,195,1.609124e+09,26900.50,27272.84,27066.07,27269.36,821.401816
47653,196,1.609128e+09,26820.00,27297.00,27265.66,26846.52,825.431767
47654,197,1.609132e+09,26800.00,27135.00,26843.75,26927.17,487.503626
